In [1]:
import pandas as pd
from utils import process_dataframe
from utils import function
import itertools
import time

In [2]:
def generate_substitution_matrices(symbols, range_vals):
    pairs = list(itertools.product(symbols, repeat=2))
    value_combinations = list(itertools.product(range_vals, repeat=len(pairs)))
    
    matrices = []
    for values in value_combinations:
        matrix = {pair: value for pair, value in zip(pairs, values)}
        matrices.append(matrix)
    
    return matrices


In [5]:
def concatenar_coluna_para_string(df, nome_coluna):
    # Verificar se a coluna existe no dataframe
    if nome_coluna not in df.columns:
        return "Coluna não encontrada."

    # Concatenar todos os itens da coluna em uma string
    string_concatenada = ''.join(df[nome_coluna].astype(str))

    return string_concatenada

In [7]:
def igualar_tamanho(df1, df2):
    if len(df1) > len(df2):
        df1=df1.head(-1)
    elif len(df2) > len(df1):
        df2=df2.head(-1)
    else:
        a=1
    return df1, df2

In [9]:
!pwd

/c/Users/lucas/Documents/SentimentAnalysisInMentalHealth


In [11]:
df=pd.read_excel("ResultadoFineTuning.xlsx")

In [13]:
simbols=[]
for index,row in df.iterrows():
    if row["result_fine_tuning_text"]=="neutro":
        simbols.append('n')
    elif row["result_fine_tuning_text"]=="positivo":
        simbols.append('p')
    else:
        simbols.append('z')

In [15]:
df["simbol_sentiment"]=simbols

In [17]:
symbols = ['p', 'n', 'z']

# Intervalo de valores possíveis para as penalidades
range_vals = list(range(-1, 2))  # [-2, -1, 0, 1, 2]

# Gerar todas as possíveis matrizes de substituição
all_matrices = generate_substitution_matrices(symbols, range_vals)

In [23]:
len(all_matrices)

19683

In [25]:
for i in range(-1, 2):
    print(i)

-1
0
1


In [27]:
start_time = time.time()
count=1
df_output=pd.DataFrame(columns=['sessao', 'itens_paciente', 'itens_terapeuta','score_ultimo','score_max','matriz'])
for sessao  in ['SPCOMVC_T010_P001_09052020']:#in df.sessao.unique():#
    print(sessao,count)
    count=+1
    df_sessao=df.loc[df["sessao"]== sessao]
    sentimentos=["p","n","z"]
    sumarizacao_sessao_paciente=function.processar_dataframe(df_sessao,"Falante","simbol_sentiment")
    sumarizacao_sessao_terapeuta=function.processar_dataframe(df_sessao,"Falante","simbol_sentiment")
    df_paciente=sumarizacao_sessao_paciente[sumarizacao_sessao_paciente["Categoria"]=="p"]
    df_terapeuta=sumarizacao_sessao_terapeuta[sumarizacao_sessao_terapeuta["Categoria"]=="t"]
    df_paciente,df_terapeuta=igualar_tamanho(df_paciente,df_terapeuta)
    combinacoes_itens = [(p, t) for p in ["Primeiro_valor_B", "Ultimo_valor_B", "Mais_repetido_B"] 
                                   for t in ["Primeiro_valor_B", "Ultimo_valor_B", "Mais_repetido_B"]]
        # Loop sobre as combinações de itens_paciente e itens_terapeuta
    for itens_paciente, itens_terapeuta in combinacoes_itens:
        dataframe_serie = pd.DataFrame(columns=['Paciente', 'Terapeuta'])
        dataframe_serie['Paciente']=df_paciente[itens_paciente].values
        dataframe_serie['Terapeuta']=df_terapeuta[itens_terapeuta].values
        string_paciente=concatenar_coluna_para_string(df_paciente, itens_paciente)
        string_terapeuta=concatenar_coluna_para_string(df_terapeuta, itens_terapeuta)
        for matriz in  all_matrices:
            score_last,score_max=function.needleman_wunsch_matriz(seq1=string_paciente, seq2= string_terapeuta,gap=-1,substitution_matrix=matriz)
            df_output = pd.concat([df_output,pd.DataFrame.from_dict({'sessao': [sessao], 
                                                                             'itens_paciente': [itens_paciente], 
                                                                             'itens_terapeuta': [itens_terapeuta], 
                                                                             'score_ultimo':[score_last],
                                                                             'score_max':[score_max],
                                                                             'matriz':[matriz]
                                                                            })])
end_time = time.time()

SPCOMVC_T010_P001_09052020 1


In [28]:
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

Tempo de execução: 1014.5217974185944 segundos


In [ ]:
df_output

In [ ]:
1+1

In [ ]:
df_output_unique=df_output[["sentimento paciente",
           "itens_paciente",
           "sentimento terapeuta","itens_terapeuta",	
           "window",
           "winc",
           "lag"]].drop_duplicates()

In [ ]:
df_result.head()

In [ ]:
correl_max_mean_list_paciente=[]
p_valor_max_mean_list_paciente=[]
correl_max_mean_list_terapeuta=[]
p_valor_max_mean_list_terapeuta=[]
correl_min_mean_list_paciente=[]
p_valor_min_mean_list_paciente=[]
correl_min_mean_list_terapeuta=[]
p_valor_min_mean_list_terapeuta=[]
correl_max_median_list_paciente=[]
p_valor_max_median_list_paciente=[]
correl_max_median_list_terapeuta=[]
p_valor_max_median_list_terapeuta=[]
correl_min_median_list_paciente=[]
p_valor_min_median_list_paciente=[]
correl_min_median_list_terapeuta=[]
p_valor_min_median_list_terapeuta=[]
for index,row in df_output_unique.iterrows():
    df_result_comb=df_output[(df_output["sentimento paciente"]==row["sentimento paciente"])&
                            (df_output["itens_paciente"]==row["itens_paciente"])&
                            (df_output["sentimento terapeuta"]==row["sentimento terapeuta"])&
                            (df_output["itens_terapeuta"]==row["itens_terapeuta"])&
                            (df_output["window"]==row["window"])&
                            (df_output["winc"]==row["winc"])&
                            (df_output["lag"]==row["lag"])
                            ]
    correl_max_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiTerapeuta')[0])
    p_valor_max_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiTerapeuta')[1])
    correl_max_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiPaciente')[0])
    p_valor_max_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiPaciente')[1])
    correl_min_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiTerapeuta')[0])
    p_valor_min_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiTerapeuta')[1])
    correl_min_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiPaciente')[0])
    p_valor_min_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiPaciente')[1])
    correl_max_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiTerapeuta')[0])
    p_valor_max_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiTerapeuta')[1])
    correl_max_median_list_paciente.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiPaciente')[0])
    p_valor_max_median_list_paciente.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiPaciente')[1])
    correl_min_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiTerapeuta')[0])
    p_valor_min_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiTerapeuta')[1])
    correl_min_median_list_paciente.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiPaciente')[0])
    p_valor_min_median_list_paciente.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiPaciente')[1])
    #print(comb,df_filtrado_max_mean_T['max_mean'].corr(df_filtrado_max_mean_T['WaiTerapeuta']))